In [45]:
import os 
import sys

import sklearn as sk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
import xgboost as xgb

from scipy.stats import kurtosis, entropy
from scipy.fft import fft

sys.path.append(r"c:\Users\keng-tse\Desktop\csi_tool") #這行為絕對路徑，如需使用，必須要修改為當前決路徑
import csi_tool
import denoise 

plt.rcParams.update({
    'font.size': 16,  # 調整全局字體大小
    'axes.titlesize': 20,  # 圖表標題的字體大小
    'axes.labelsize': 18,  # X 和 Y 軸標籤字體大小
    # 'xtick.labelsize': 14,  # X 軸刻度字體大小
    # 'ytick.labelsize': 14,  # Y 軸刻度字體大小
    # 'legend.fontsize': 16,  # 圖例字體大小
    # 'figure.titlesize': 22  # 整個圖表標題的字體大小
})

def load_data(reference_points):
    data = []          # 用于存储所有数据
    rp_labels = []     # 用于存储参考点编号标签
    coord_labels = []  # 用于存储 (x, y) 坐标标签

    for path, (ref_id, coord) in reference_points.items():
        df = pd.read_excel(path)
        data.append(df.values)
        rp_labels.extend([ref_id] * len(df))
        coord_labels.extend([coord] * len(df))
    data = pd.DataFrame(np.vstack(data))
    rp_labels = pd.Series(rp_labels, name="Reference Point ID")
    coord_labels = pd.DataFrame(coord_labels, columns=["X", "Y"])
    return data, rp_labels, coord_labels

def save_individual_cfr_plots_by_label(data, rp_labels, save_dir=r"C:\Users\keng-tse\Desktop\csi-plot\csi20Hz_1222\amp_std"):
    # Create directory to save files if it does not exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Separate CFR data (first 48 columns)
    #CFR_data = data.iloc[:, :48]

    # Iterate through each reference point ID and save a plot
    unique_ref_ids = rp_labels.unique()
    for ref_id in unique_ref_ids:
        # Select CFR data corresponding to the current reference point ID
        ref_data = data[rp_labels == ref_id]
        
        # Create a plot for the reference point
        plt.figure(figsize=(10, 6))

        # Plot each sample (line plot for each packet)
        for i in range(ref_data.shape[0]):
            plt.plot(ref_data[i, :], label=f"Sample {i+1}", alpha=0.6)

        plt.title(f"CFR Data for Reference Point {ref_id}")
        plt.xlabel("Subcarrier Index (1-48)")
        plt.ylabel("CFR Value")
        plt.grid(True)
        # plt.legend(title="Samples", bbox_to_anchor=(1.05, 1), loc='upper left')

        # Define the file name using the reference point ID
        file_name = f"{save_dir}/ref_point_{ref_id}.png"
        
        # Save the plot as a PNG image
        plt.savefig(file_name, bbox_inches="tight")
        plt.close()  # Close the plot to free up memory
        print(f"Saved CFR plot for Reference Point {ref_id} to {file_name}")


def calculate_total_energy(amp_matrix):
    """
    Calculate the total energy across all subcarriers for each packet.

    Parameters:
        amp_matrix (numpy.ndarray): Matrix of amplitude values with shape (num_packets, num_subcarriers).

    Returns:
        numpy.ndarray: Total energy for each packet, shape (num_packets,).
    """
    return np.sum(amp_matrix, axis=1), np.std(amp_matrix, axis=1)

def save_total_energy_plots_by_label(amp, rssi, rp_labels, save_dir=r"C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1222\\total_energy", db=False):
    """
    Save plots of total subcarrier energy by packet index for each unique label.

    Parameters:
        amp (pd.DataFrame): DataFrame containing amplitude values for each subcarrier.
        rssi (pd.Series): Series containing RSSI values for each packet.
        rp_labels (pd.Series): Series containing labels for each packet.
        save_dir (str): Directory to save the plots.
        db (bool): Whether to convert total energy to dB scale.
    """
    import os
    import numpy as np
    import matplotlib.pyplot as plt

    # Create directory to save files if it does not exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Calculate total energy for each packet
    amp_matrix = amp.values
    total_energy, _ = calculate_total_energy(amp_matrix)

    # Convert to dB if specified
    if db:
        total_energy = 10 * np.log10(total_energy + 1e-10)

    # Iterate through each unique label and save a plot
    unique_labels = rp_labels.unique()
    for label in unique_labels:
        # Select data corresponding to the current label
        label_indices = rp_labels == label
        label_total_energy = total_energy[label_indices]
        label_rssi = rssi[label_indices]

        # Create a plot for the label
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plot total energy on the left y-axis
        ax1.plot(range(len(label_total_energy)), label_total_energy, marker='o', color='b', alpha=0.7, label="Total Energy")
        ax1.set_xlabel("Packet Index")
        ax1.set_ylabel("Total Energy", color='b')
        ax1.tick_params(axis='y', labelcolor='b')
        ax1.grid(True)

        # Create a second y-axis for RSSI
        ax2 = ax1.twinx()
        ax2.plot(range(len(label_rssi)), label_rssi, marker='*', color='r', alpha=0.7, label="RSSI")
        ax2.set_ylabel("RSSI", color='r')
        ax2.tick_params(axis='y', labelcolor='r')

        # Title and legend
        plt.title(f"Total Subcarrier Energy and RSSI for Label {label}")
        fig.tight_layout()

        # Define the file name using the label
        file_name = f"{save_dir}/label_{label}_total_energy.png"

        # Save the plot as a PNG image
        plt.savefig(file_name, bbox_inches="tight")
        plt.close()  # Close the plot to free up memory
        print(f"Saved total energy plot for Label {label} to {file_name}")


In [7]:
base_path = r"C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1223_phone\5G\20Mhz\csv\all_no_sub_clean"
reference_points = {}

In [9]:
# 每個點的實際位置 (x, y)，基於 0.6m 的間距
spacing = 0.6  # 每隔 0.6m
coordinates = {
    # 下邊界 (1-10 和 40-31)
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),

    # 左邊界 (1-11)
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),

    # 上邊界 (11-21)
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),

    # 右邊界 (21-31)
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),

    # 中間點 (41-49)
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

# 生成 reference_points 字典
for ref_id, coord in coordinates.items():
    folder_path = os.path.join(base_path, f"reference_point_{ref_id}.xlsx")
    reference_points[folder_path] = (ref_id, coord)

# 打印結果
for path, (ref_id, coord) in reference_points.items():
    print(f"Path: {path}, ID: {ref_id}, Coordinates: {coord}")

Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1223_phone\5G\20Mhz\csv\all_no_sub_clean\reference_point_1.xlsx, ID: 1, Coordinates: (0, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1223_phone\5G\20Mhz\csv\all_no_sub_clean\reference_point_40.xlsx, ID: 40, Coordinates: (0.6, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1223_phone\5G\20Mhz\csv\all_no_sub_clean\reference_point_39.xlsx, ID: 39, Coordinates: (1.2, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1223_phone\5G\20Mhz\csv\all_no_sub_clean\reference_point_38.xlsx, ID: 38, Coordinates: (1.8, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1223_phone\5G\20Mhz\csv\all_no_sub_clean\reference_point_37.xlsx, ID: 37, Coordinates: (2.4, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1223_phone\5G\20Mhz\csv\all_no_sub_clean\reference_point_36.xlsx, ID: 36, Coordinates: (3.0,

In [11]:
data, rp_labels, coord_labels = load_data(reference_points)

In [12]:
print(np.array(data).shape)

(24451, 130)


In [55]:
amp = data.iloc[:,:64]
rssi = data.iloc[:,-2]

In [57]:
print(rssi)

0       -49.0
1       -50.0
2       -51.0
3       -50.0
4       -49.0
         ... 
24446   -56.0
24447   -59.0
24448   -58.0
24449   -56.0
24450   -58.0
Name: 128, Length: 24451, dtype: float64


In [59]:
print("Indices of rp_labels:", rp_labels.index)
print("Indices of rssi:", rssi.index)
print("Indices of amp:", amp.index)

Indices of rp_labels: RangeIndex(start=0, stop=24451, step=1)
Indices of rssi: RangeIndex(start=0, stop=24451, step=1)
Indices of amp: RangeIndex(start=0, stop=24451, step=1)


In [61]:
save_total_energy_plots_by_label(amp, rssi, rp_labels, save_dir=r"C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64")

Saved total energy plot for Label 1 to C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64/label_1_total_energy.png
Saved total energy plot for Label 40 to C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64/label_40_total_energy.png
Saved total energy plot for Label 39 to C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64/label_39_total_energy.png
Saved total energy plot for Label 38 to C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64/label_38_total_energy.png
Saved total energy plot for Label 37 to C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64/label_37_total_energy.png
Saved total energy plot for Label 36 to C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64/label_36_total_energy.png
Saved total energy plot for Label 35 to C:\\Users\\keng-tse\\Desktop\\csi-plot\\csi20Hz_1223\\rssi_csi_total_energy_64/label_35_tota